In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Importing necessary dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import plotly.express as px

warnings.filterwarnings("ignore")

from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler

!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

from sklearn.pipeline import Pipeline


from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

plt.style.use('fivethirtyeight')

from plotly.offline import download_plotlyjs,init_notebook_mode
init_notebook_mode(connected=True)

#  2. Data Preprocessing 

In [ ]:
train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')


In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.apply(lambda x:len(x.unique()))

In [ ]:
train.isnull().sum()

In [ ]:
cat_col = []
for x in train.dtypes.index:
    if train.dtypes[x] == "object":
        cat_col.append(x)
        
cat_col

In [ ]:
cat_col.remove('gender')
cat_col

In [ ]:
for col in cat_col:
    print(col)
    print(train[col].value_counts())
    print()

# 3. Visualizing the Data 

In [ ]:

boxplot = train.boxplot(column ='city_development_index')

In [ ]:
train.gender.value_counts()

In [ ]:
print("Number of Missing Values: ", train['gender'].isna().sum())
fig = px.pie(train['gender'].value_counts(), values='gender', names = train['gender'].value_counts().index,title = 'gender',template='ggplot2')
fig.show()

In [ ]:
print("Number of Missing Values: ", train['relevent_experience'].isna().sum())
fig = px.pie(train['relevent_experience'].value_counts(), values='relevent_experience', 
             names = train['relevent_experience'].value_counts().index,title = 'relevent_experience',template='ggplot2')
fig.show()


In [ ]:
print ("no.of missing values:" ,train["education_level"].isna().sum())
fig = px.pie(train['education_level'].value_counts(),values = "education_level",
            names = train['education_level'].value_counts().index,title = "education_level",template = 'ggplot2')
fig.show()

In [ ]:
print ('no.of values missing',train["major_discipline"].isna().sum())
fig = px.pie(train['major_discipline'].value_counts(),values = 'major_discipline', names = train["major_discipline"].value_counts().index,title = "major_discipline", template = "ggplot2")
fig.show()

In [ ]:
print ("no.of values missing:",train["experience"].isna().sum())
fig = px.pie(train["experience"].value_counts(), values = "experience", names = train["experience"].value_counts().index , template = "ggplot2",title = "experience")
fig.show()

In [ ]:
print ("no.of missing values:",train["company_size"].isna().sum())
fig = px.pie(train["company_size"].value_counts(), values = "company_size", names = train["company_size"].value_counts().index ,title = "company_size", template = "ggplot2")
fig.show()

In [ ]:
print ("no.of missing values:",train["company_type"].isna().sum())
fig = px.pie(train["company_type"].value_counts(), values = "company_type", names = train["company_type"].value_counts().index ,title = "company_type", template = "ggplot2")
fig.show()

In [ ]:
print ("no.of missing values:",train["last_new_job"].isna().sum())
fig = px.pie(train["last_new_job"].value_counts(), values = "last_new_job", names = train["last_new_job"].value_counts().index ,title = "last_new_job", template = "ggplot2")
fig.show()

In [ ]:
print ("no.ofmissing values:",train["training_hours"].isna().sum())
train["training_hours"].describe()
train.boxplot(column = 'training_hours')

In [ ]:
print ("no.of missing values:",train["target"].isna().sum())
fig = px.pie(train["target"].value_counts(), values = "target", names = train["target"].value_counts().index ,title = "target", template = "ggplot2")
fig.show()

4. Checking the null imputation in the data**** 

In [ ]:
train_mice = train.copy()

In [ ]:
for feature in train_mice.columns:
    print('*******','Column name:',feature,'*******')
    print(train_mice[feature].unique())
    print('***********-end-***********')
    print(' ')

# 5.Data Mapping 

In [ ]:
train_mice["enrolled_university"] = train_mice["enrolled_university"].map({'no_enrollment':int(-1) , 'Full time course': int(1) , 'Part time course': int(0)})

In [ ]:
train_mice['gender'] = train_mice['gender'].map({'other':int(-1), 'Male':int(0), 'Female':int(1)})

In [ ]:
train_mice["education_level"] = train_mice["education_level"].map({'graduate': int(0),"Masters": int (1) , "High school" : int(-1) , "Phd" : int(2) , "Primary school": int(-2)})

In [ ]:
train_mice["major_discipline"] = train_mice["major_discipline"].map({"No_Major": int(-3), "Other":int(-2) , "Humanities":int(-1), "Arts": int(0) , "Business Degree": int(1), "STEM": int(2)})

In [ ]:
train_mice["company_type"] = train_mice["company_type"].map({"Pvt Ltd":int(3),"Funded Startup" : int(2), 'Early Stage Startup': int(1), 'Other': int(-1),
 'Public Sector': int(0), 'NGO' : int(-2)})

In [ ]:
train_mice["experience"] = train_mice["experience"].map({'<1': int(-3), "2": int(-2), "3":int(-2), "4": int(-2), "5": int(-1), "6" :int(-1), "7" :int(-1) , "8 ":int(0) , "9" :int(0) , "10" : int(0) , "11" : int(1) , "12" : int(1), "13" : int(1) , "14" : int(2),"15 " : int(2) , "16" : int(2), "17" : int(3),"18":int(3),"19":int(3),">20":int(-3)})

In [ ]:
train_mice['last_new_job'] = train_mice['last_new_job'].map({'1':int(-1), '>4':int(0), 'never':int(1), '4':int(4), '3':int(3), '2':int(2)})


In [ ]:
train_mice['company_size'] = train_mice['company_size'].map({'50-99':int(-1), '<10':int(-1), '10000+':int(0), '5000-9999':int(0), 
                                                   '1000-4999':int(1), '10/49':int(1), '100-500':int(2),'500-999':int(2)})

In [ ]:
train_mice.isnull().sum()

# 6. Using MICE for missing values imputations 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

mice_imputer = IterativeImputer()

In [ ]:
train_mice["gender"] = mice_imputer.fit_transform(train_mice[['gender']])
train_mice['gender'] = round(train_mice['gender'])
train_mice["enrolled_university"] = mice_imputer.fit_transform(train_mice[['enrolled_university']])
train_mice['enrolled_university'] = round(train_mice['enrolled_university'])
train_mice["education_level"] = mice_imputer.fit_transform(train_mice[['education_level']])
train_mice['education_level'] = round(train_mice['education_level'])
train_mice["major_discipline"] = mice_imputer.fit_transform(train_mice[['major_discipline']])
train_mice['major_discipline'] = round(train_mice['major_discipline'])
train_mice["experience"] = mice_imputer.fit_transform(train_mice[['experience']])
train_mice['experience'] = round(train_mice['experience'])
train_mice["company_size"] = mice_imputer.fit_transform(train_mice[['company_size']])
train_mice['company_size'] = round(train_mice['company_size'])
train_mice["company_type"] = mice_imputer.fit_transform(train_mice[['company_type']])
train_mice['company_type'] = round(train_mice['company_type'])
train_mice["last_new_job"] = mice_imputer.fit_transform(train_mice[['last_new_job']])
train_mice['last_new_job'] = round(train_mice['last_new_job'])


In [ ]:
train_mice['city'] = pd.Categorical(train_mice["city"]).codes
train_mice['relevent_experience'] = pd.Categorical(train_mice["relevent_experience"]).codes

In [ ]:
train_mice.isnull().sum()

In [ ]:
test["enrolled_university"] = test["enrolled_university"].map({'no_enrollment':int(-1) , 'Full time course': int(1) , 'Part time course': int(0)})
test["education_level"] = test["education_level"].map({'graduate': int(0),"Masters": int (1) , "High school" : int(-1) , "Phd" : int(2) , "Primary school": int(-2)})
test["major_discipline"] = test["major_discipline"].map({"No_Major": int(-3), "Other":int(-2) , "Humanities":int(-1), "Arts": int(0) , "Business Degree": int(1), "STEM": int(2)})
test["company_type"] = test["company_type"].map({"Pvt Ltd":int(3),"Funded Startup" : int(2), 'Early Stage Startup': int(1), 'Other': int(-1),
 'Public Sector': int(0), 'NGO' : int(-2)})
test["experience"] = test["experience"].map({'<1': int(-3), "2": int(-2), "3":int(-2), "4": int(-2), "5": int(-1), "6" :int(-1), "7" :int(-1) , "8 ":int(0) , "9" :int(0) , "10" : int(0) , "11" : int(1) , "12" : int(1), "13" : int(1) , "14" : int(2),"15 " : int(2) , "16" : int(2), "17" : int(3),"18":int(3),"19":int(3),">20":int(-3)})

test['company_size'] = test['company_size'].map({'50-99':int(-1), '<10':int(-1), '10000+':int(0), '5000-9999':int(0), 
                                                   '1000-4999':int(1), '10/49':int(1), '100-500':int(2),'500-999':int(2)})
test['last_new_job'] = test['last_new_job'].map({'1':int(-1), '>4':int(0), 'never':int(1), '4':int(4), '3':int(3), '2':int(2)})
test['gender'] = test['gender'].map({'other':int(-1), 'Male':int(0), 'Female':int(1)})

In [ ]:
test['city'] = pd.Categorical(test["city"]).codes
test['relevent_experience'] = pd.Categorical(test["relevent_experience"]).codes

In [ ]:
test["gender"] = mice_imputer.fit_transform(test[['gender']])
test['gender'] = round(test['gender'])
test["enrolled_university"] = mice_imputer.fit_transform(test[['enrolled_university']])
test['enrolled_university'] = round(test['enrolled_university'])
test["education_level"] = mice_imputer.fit_transform(test[['education_level']])
test['education_level'] = round(test['education_level'])
test["major_discipline"] = mice_imputer.fit_transform(test[['major_discipline']])
test['major_discipline'] = round(test['major_discipline'])
test["experience"] = mice_imputer.fit_transform(test[['experience']])
test['experience'] = round(test['experience'])
test["company_size"] = mice_imputer.fit_transform(test[['company_size']])
test['company_size'] = round(test['company_size'])
test["company_type"] = mice_imputer.fit_transform(test[['company_type']])
test['company_type'] = round(test['company_type'])
test["last_new_job"] = mice_imputer.fit_transform(test[['last_new_job']])
test['last_new_job'] = round(test['last_new_job'])


In [ ]:
test.isnull().sum()

# 7. Initial data split 

In [ ]:
X_train = train_mice[['city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours']]


train_labels = train_mice['target']

In [ ]:
X_test =test[[ 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours']]

In [ ]:
test_labels = np.load('/kaggle/input/job-change-answers/jobchange_test_target_values.npy')

7. Pipelining 

In [ ]:
rf_pipe = Pipeline(steps =[ ('std_scale',StandardScaler()), ("RF",RandomForestClassifier(random_state=0,max_depth= 10, max_features= 5,min_samples_leaf= 30, min_samples_split= 100, n_estimators= 500))])
rf_pipe.fit(X_train,train_labels)

dt_pipe = Pipeline(steps =[ ('_std_scale',StandardScaler()), ("DT",DecisionTreeClassifier(criterion='gini',max_features=10, max_depth=10, min_samples_leaf=15, min_samples_split=100,random_state=0)) ])
dt_pipe.fit(X_train,train_labels)

In [ ]:
rf_pipe.fit(X_train,train_labels)

In [ ]:
dt_pipe.fit(X_train,train_labels)

8. Determining the importance of factors for predictions 

In [ ]:
feature_importances = pd.Series(rf_pipe.steps[1][1].feature_importances_,index =X_train.columns);
feature_importances.nlargest(15).plot(kind='barh');
plt.title('Random forest features_importances');

In [ ]:
rf_train_predict = rf_pipe.predict(X_train)
rf_test_predict = rf_pipe.predict(X_test)

dt_train_predict = dt_pipe.predict(X_train)
dt_test_predict = dt_pipe.predict(X_test)

# 9. Random Forest and Decision Tree

In [ ]:
print('Random Forest classification_report on test_set')
print(classification_report(test_labels,rf_test_predict))

In [ ]:
print('Random Forest classification_report on train_set')
print(classification_report(train_labels,rf_train_predict))

In [ ]:
print('Decision Tree classification_report on test_set')
print(classification_report(test_labels,dt_test_predict))

In [ ]:
print('Decision Tree classification_report on train_set')
print(classification_report(train_labels,dt_train_predict))

In [ ]:
rf_probs_train = rf_pipe.predict_proba(X_train)
rf_probs_train = rf_probs_train[:, 1]

rf_probs_test = rf_pipe.predict_proba(X_test)
rf_probs_test = rf_probs_test[:, 1]


dt_probs_train = dt_pipe.predict_proba(X_train)
dt_probs_train = dt_probs_train[:, 1]

dt_probs_test = dt_pipe.predict_proba(X_test)
dt_probs_test = dt_probs_test[:, 1]

In [ ]:
auc_RF_train = roc_auc_score(train_labels,rf_probs_train)
auc_RF_test = roc_auc_score(test_labels,rf_probs_test)
print("Random forest auc on train set",auc_RF_train)
print("Random forest auc on test set", auc_RF_test)

In [ ]:
auc_dt_train = roc_auc_score(train_labels,dt_probs_train)
auc_dt_test = roc_auc_score(test_labels,dt_probs_test)
print("Decision tree auc on train set",auc_dt_train)
print("Decision tree auc on test set", auc_dt_test)

10. Predictions 

In [ ]:
predict = rf_pipe.predict_proba(X_test)
predict = predict[:,1]

11.Creating csv submission file

In [ ]:
submission = pd.DataFrame({'enrollee_id':test['enrollee_id'],'target':predict})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)